In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [2]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(83000, 502) (21975, 502)


In [3]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [4]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float().squeeze()
train_X = torch.from_numpy(train.drop([0], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float().squeeze()
val_X = torch.from_numpy(val.drop([0], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float().squeeze()
test_X = torch.from_numpy(test.drop([0], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10987])


In [5]:
import torch.nn as nn
import numpy as np

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, trial,vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_dim = trial.suggest_int("hidden_dim",16,256)
        embedding_dim = trial.suggest_int("embedding_dim",16,128)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [6]:
data_dir = './lstm-amazon-hyperband-trails_6/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

    filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./lstm-amazon-hyperband-trails_6/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [7]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np

epochs = 10
vocab_size=10000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define an objective function to be minimized.
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #epochs = trial.suggest_int("epochs",5,15)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    model = LSTMClassifier(trial, vocab_size).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    loss_fn = torch.nn.BCELoss()
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_accuracy_epoch = []
    time_train = []
    final_val_acc = 0
    for epoch in range(epochs):
        model.train()
        start = time.time()
        total_loss = 0
        train_acc = 0
        total = 0
        correct = 0
        for batch in train_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            optimizer.zero_grad()
            prediction = model(batch_X)
            loss = loss_fn(prediction, batch_y)
            loss.backward()
            optimizer.step()
            result = np.round(prediction.detach().cpu())
            total_loss += loss.data.item()
            total += batch_y.size(0)
            correct += (result == batch_y.cpu()).sum().item()
            train_acc = correct/total
        train_loss_epoch.append(np.round(total_loss / len(train_dl), 3))
        train_acc_epoch.append(np.round(train_acc*100,3))
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_dl)))
        with torch.no_grad():
            model.eval()
            correct = 0
            total = 0
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                prediction = model(inputs_val)
                loss = loss_fn(prediction, labels_val)
                val_loss.append(np.round(loss.item(),3))
                result = np.round(prediction.cpu())
                total += labels_val.size(0)
                correct += (result == labels_val.cpu()).sum().item()
            val_accuracy_epoch.append(np.round((correct/total)*100, 3))
            val_loss_epoch.append(np.round(np.mean(val_loss),3))
            end = time.time() - start
            final_val_acc = np.round((correct/total)*100, 3)
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(correct/total))
            time_train.append(np.round(end,3))
    write_to_csv(trial.number, epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_accuracy_epoch, time_train)
    return final_val_acc


In [8]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [9]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=6,callbacks=[callback])


[I 2022-05-14 14:17:31,807] A new study created in memory with name: no-name-be633316-4d7f-46a1-8864-63826046c124


Epoch: 0, BCELoss: 0.5461156216012426
Val Loss: 0.523 	Val Acc: 0.763
Epoch: 1, BCELoss: 0.51135969770601
Val Loss: 0.499 	Val Acc: 0.774
Epoch: 2, BCELoss: 0.48986808676497046
Val Loss: 0.478 	Val Acc: 0.785
Epoch: 3, BCELoss: 0.46383242201374236
Val Loss: 0.442 	Val Acc: 0.802
Epoch: 4, BCELoss: 0.4155064238392445
Val Loss: 0.383 	Val Acc: 0.829
Epoch: 5, BCELoss: 0.37122985979160633
Val Loss: 0.346 	Val Acc: 0.846
Epoch: 6, BCELoss: 0.3404575882816171
Val Loss: 0.325 	Val Acc: 0.857
Epoch: 7, BCELoss: 0.3209754641874727
Val Loss: 0.312 	Val Acc: 0.863
Epoch: 8, BCELoss: 0.3075996730701033
Val Loss: 0.304 	Val Acc: 0.869
Epoch: 9, BCELoss: 0.2976961928543197


[I 2022-05-14 14:25:17,584] Trial 0 finished with value: 87.313 and parameters: {'optimizer': 'SGD', 'lr': 0.010874802693711393, 'dropout': 0.5682104325709664, 'hidden_dim': 254, 'embedding_dim': 116}. Best is trial 0 with value: 87.313.


Val Loss: 0.299 	Val Acc: 0.873
Epoch: 0, BCELoss: 0.3199236743960036
Val Loss: 0.262 	Val Acc: 0.887
Epoch: 1, BCELoss: 0.23691224419478193
Val Loss: 0.260 	Val Acc: 0.889
Epoch: 2, BCELoss: 0.19640724478536342
Val Loss: 0.280 	Val Acc: 0.888
Epoch: 3, BCELoss: 0.1574494143947959
Val Loss: 0.320 	Val Acc: 0.887
Epoch: 4, BCELoss: 0.12372976724305246
Val Loss: 0.368 	Val Acc: 0.882
Epoch: 5, BCELoss: 0.10589253042476723
Val Loss: 0.389 	Val Acc: 0.883
Epoch: 6, BCELoss: 0.08658481236625777
Val Loss: 0.434 	Val Acc: 0.877
Epoch: 7, BCELoss: 0.07746649844844052
Val Loss: 0.473 	Val Acc: 0.877
Epoch: 8, BCELoss: 0.07079962222256238
Val Loss: 0.488 	Val Acc: 0.876
Epoch: 9, BCELoss: 0.06816943707533675


[I 2022-05-14 14:31:32,432] Trial 1 finished with value: 87.787 and parameters: {'optimizer': 'Adam', 'lr': 0.0034398882685020165, 'dropout': 0.35923198862861494, 'hidden_dim': 182, 'embedding_dim': 106}. Best is trial 1 with value: 87.787.


...

Val Loss: 0.294 	Val Acc: 0.886
Epoch: 0, BCELoss: 0.5923364323126264
Val Loss: 0.519 	Val Acc: 0.762
Epoch: 1, BCELoss: 0.4909926176430231
Val Loss: 0.471 	Val Acc: 0.772
Epoch: 2, BCELoss: 0.4557159566915179
Val Loss: 0.445 	Val Acc: 0.785
Epoch: 3, BCELoss: 0.43373278821089184
Val Loss: 0.427 	Val Acc: 0.797
Epoch: 4, BCELoss: 0.4155030656561076
Val Loss: 0.411 	Val Acc: 0.810
Epoch: 5, BCELoss: 0.398856950901359
Val Loss: 0.395 	Val Acc: 0.822
Epoch: 6, BCELoss: 0.38297326900513773
Val Loss: 0.381 	Val Acc: 0.832
Epoch: 7, BCELoss: 0.36760327456945396
Val Loss: 0.366 	Val Acc: 0.840
Epoch: 8, BCELoss: 0.3526868577193783
Val Loss: 0.352 	Val Acc: 0.847
Epoch: 9, BCELoss: 0.33878418424402373


[I 2022-05-14 14:44:55,261] Trial 4 finished with value: 85.202 and parameters: {'optimizer': 'RMSprop', 'lr': 2.0204359256865862e-05, 'dropout': 0.26229236255869115, 'hidden_dim': 94, 'embedding_dim': 95}. Best is trial 3 with value: 88.633.


Val Loss: 0.340 	Val Acc: 0.852
Epoch: 0, BCELoss: 23.175059481952562
Val Loss: 23.198 	Val Acc: 0.766
Epoch: 1, BCELoss: 23.24055723282228
Val Loss: 23.194 	Val Acc: 0.767
Epoch: 2, BCELoss: 23.270304251291666
Val Loss: 23.337 	Val Acc: 0.766
Epoch: 3, BCELoss: 23.512851976486573
Val Loss: 23.690 	Val Acc: 0.763
Epoch: 4, BCELoss: 23.57066024872194
Val Loss: 23.717 	Val Acc: 0.763
Epoch: 5, BCELoss: 23.57212242563087
Val Loss: 23.722 	Val Acc: 0.763
Epoch: 6, BCELoss: 23.56920702486153
Val Loss: 23.727 	Val Acc: 0.763
Epoch: 7, BCELoss: 23.573567312309542
Val Loss: 23.725 	Val Acc: 0.763
Epoch: 8, BCELoss: 23.572156287963132
Val Loss: 23.711 	Val Acc: 0.763
Epoch: 9, BCELoss: 23.594468510294536


[I 2022-05-14 14:52:19,366] Trial 5 finished with value: 76.265 and parameters: {'optimizer': 'Adam', 'lr': 0.815005993880748, 'dropout': 0.14754807477079482, 'hidden_dim': 247, 'embedding_dim': 55}. Best is trial 3 with value: 88.633.


Val Loss: 23.729 	Val Acc: 0.763


In [10]:
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  6
  Number of complete trials:  6
Best trial:
  Value:  88.633
  Params: 
    optimizer: Adam
    lr: 0.00021848264157464222
    dropout: 0.5288820827983397
    hidden_dim: 106
    embedding_dim: 67


In [11]:
data_dir = './lstm-amazon-hyperband/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

torch.save(best_model.state_dict(),"./lstm-amazon-hyperband/lstm_amazon_hyperband.pth")

In [12]:
def test(model, test_dl, epochs):
    model.eval()
    correct = 0
    total = 0
#     results = []
#     labels = []
    with torch.no_grad():
        for batch in test_dl:         
            batch_X, batch_y = batch
            batch_X = batch_X.to(device)
            prediction = model(batch_X)
            result = np.round(prediction.cpu())
#             results.extend(list(result.numpy()))
#             labels.extend(list(batch_y.numpy()))
            total += batch_y.size(0)
            correct += (result == batch_y).sum().item()
    print("Accuracy:", correct/total)

In [13]:
test(best_model, test_dl, epochs)

/ext3/miniconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:691: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:925.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Accuracy: 0.8833166469463912
